# Evaluation Model 6
## Model name : unsloth/mistral-7b-v0.3
## Link Colab : [Click](https://colab.research.google.com/drive/1qPa0zKxjyVTefuoOnfZbSMcDdIs3mcJi?usp=sharing)
## Link Adapter weight : [Click](https://drive.google.com/file/d/1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0/view?usp=drive_link)

# Download Dependencies

In [ ]:
%%capture
! pip install unsloth
! pip install rouge_score
! pip install evaluate

In [ ]:
%%capture
! pip install -U peft
! pip install bert_score

# Import Dependencies

In [ ]:
from tqdm import tqdm
import kagglehub
import os

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import evaluate

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datasets import load_dataset
import evaluate
import torch
import bert_score
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import os
import re

In [ ]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [ ]:
from peft import PeftModel

# Load dataset

In [ ]:
# 1. Load dataset
dataset = load_dataset("farwew/DoctorsAnswerTextDataset-in-Indonesian")

README.md:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

(…)-00000-of-00001-e657306e4ba4079a.parquet:   0%|          | 0.00/17.9M [00:00<?, ?B/s]

(…)-00000-of-00001-6bdfe08bb1c69cf1.parquet:   0%|          | 0.00/948k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19845 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [ ]:
# 3. Ambil data test
dataset_eval = dataset["test"]

# 4. Konversi ke pandas DataFrame
df = dataset_eval.to_pandas()
df.head()

,instruction,input,output,topic_set,topics
0,Selalu tersedak setiap makan dan minum,Selalu tersedak setiap makan dan minum. Dok sa...,batuk adalah respon tubuh untuk mengeluarkan b...,gastritis,penyakit-parkinson multiple-sclerosis gastritis
1,Apa obat untuk demam?,Apa obat untuk demam?. malam dok saya mau tany...,"kepala pusing, perut mual, punggung sakit dan ...",mual,flu pilek sakit-kepala nyeri-punggung mual
2,Cara mengatasi perut kembung pada anak,Cara mengatasi perut kembung pada anak. punya ...,perut kembung menandakan adanya penumpukan gas...,perut-kembung,anak gastritis perut-kembung
3,Perut Bagian Bawah Kembung,"Perut Bagian Bawah Kembung. saya mau bertanya,...","seperti: sayur: kol, brokoli, kembang kol, asp...",gangguan-pencernaan,gangguan-pencernaan
4,Nyeri dada menjalar ke punggung,Nyeri dada menjalar ke punggung. Malam dok tad...,saya coba bantu jawab. keluhan nyeri dada menj...,sakit-maag,nyeri-dada sakit-maag


In [ ]:
# 5. Stratified split berdasarkan kolom 'topic_set'
train_df, stratified_sample_df = train_test_split(
    df,
    test_size=100,  # ubah sesuai proporsi yang diinginkan
    stratify=df["topic_set"],
    random_state=42
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
stratified_dataset = Dataset.from_pandas(stratified_sample_df)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

# Load Adapter

In [ ]:
# %%capture
# !pip install gdown

In [ ]:
# ! gdown 'https://drive.google.com/uc?id=1sqVp1FinYwrqss-S1i6chSK2r9UN9Jx0'

In [ ]:
# ! unzip "/content/1 epoch -  Mistral_v0.3_ instruct (7B) - Alpaca (run 3).zip"

#  Notebook Evaluasi Base Model (tanpa Adapter LoRA)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.5.7: Fast Gemma2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

In [ ]:
model.eval()

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_laye

In [ ]:
alpaca_prompt = (
    "Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan "
    "dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, "
    "sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n"
    "### Keluhan:\n{}\n\n"
    "### Detail Pertanyaan:\n{}\n\n"
    "### Jawaban:\n{}"
)

In [ ]:
def clean_medical_response(text: str) -> str:
    # Bersihkan dan kapitalisasi setiap kalimat
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
        # Kapital awal
        sentence = sentence[0].upper() + sentence[1:] if len(sentence) > 1 else sentence.upper()
        # Tambahkan titik jika perlu
        if not sentence.endswith(('.', '!', '?')):
            sentence += '.'
        cleaned_sentences.append(sentence)
    return ' '.join(cleaned_sentences)

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_func(examples):
    titles  = examples["instruction"]  # atau ganti dengan "title"
    inputs  = examples["input"]
    outputs = examples["output"]

    texts = []
    for title, input_text, output_text in zip(titles, inputs, outputs):
        cleaned_output = clean_medical_response(output_text)
        full_prompt = alpaca_prompt.format(title, input_text, cleaned_output) + EOS_TOKEN
        texts.append(full_prompt)

    return { "text": texts }

## Inference dengan base model

In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "mencegah mual",
            "Anak saya mengalami mual, bagaimana mencegah mual?",
            ""
        )
    ],
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

You have set `use_cache` to `False`, but cache_implementation is set to dynamic. cache_implementation will have no effect.


Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
mencegah mual

### Detail Pertanyaan:
Anak saya mengalami mual, bagaimana mencegah mual?

### Jawaban:
Untuk mencegah mual pada anak, ada beberapa hal yang dapat dilakukan:
* Pastikan anak mendapatkan cukup istirahat.
* Berikan makanan yang ringan dan mudah dicerna, seperti roti tawar, nasi putih, atau bubur. Hindari makanan yang berlemak, pedas, atau asam.
* Pastikan anak tetap terhidrasi dengan minum banyak air putih atau jus buah.
* Hindari aktivitas fisik yang berat atau gerakan tiba-tiba.
* Jika anak sedang mengalami mual karena perjalanan, berikan obat mual yang sesuai dengan usia dan berat badan anak.
* Jika mual terus berlanjut atau disertai gejala lain seperti demam, muntah darah, atau diare, segera konsultasikan ke dokte

## Evaluation

In [ ]:
# === Evaluasi === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_instructions = []
all_inputs = []

In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r"\p{P}+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
FastLanguageModel.for_inference(model)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_laye

In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating Base Model"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating Base Model:   1%|          | 1/100 [01:43<2:50:11, 103.15s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, akibatn

Evaluating Base Model:   2%|▏         | 2/100 [04:14<3:34:22, 131.25s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:
Konstipasi dapat disebabkan oleh berbagai fakt

Evaluating Base Model:   3%|▎         | 3/100 [05:18<2:43:11, 100.94s/it]

...

Evaluating Base Model:   8%|▊         | 8/100 [15:11<3:01:26, 118.33s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
sakit perut sebelah kiri bawah

### Detail Pertanyaan:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sa

Evaluating Base Model:   9%|▉         | 9/100 [16:13<2:32:31, 100.56s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Nyeri Ulu Hati Pasca Operasi Usus buntu

### Detail Pertanyaan:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampu

Evaluating Base Model: 100%|██████████| 100/100 [2:45:03<00:00, 99.04s/it] 


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/base_model_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI BASE MODEL SELESAI ===")
print(metrics)

=== EVALUASI BASE MODEL SELESAI ===
{'bert_score': {'precision': 64.92, 'recall': 67.31, 'f1': 66.07}, 'bleu': 1.99, 'rouge': {'rouge1': np.float64(28.09), 'rouge2': np.float64(4.83), 'rougeL': np.float64(13.04)}}


In [ ]:
hasil_base = pd.read_csv(f"{output_dir}/base_model_outputs.csv")
hasil_base

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,Solusi atasi sakit maag yang tak kunjung sembu...,usus halus ataupun kerongkongan yang dapat dis...,Kamu adalah asisten medis cerdas yang membantu...,0.678952,0.687113,0.683008
1,solusi untuk mengatasi konstipasi,solusi untuk mengatasi konstipasi. dok kalo ba...,konstipasi adalah keadaan tidak dapat buang ai...,Kamu adalah asisten medis cerdas yang membantu...,0.628675,0.661004,0.644434
2,Solusi atasi sakit maag yang tak kunjung sembuh,Solusi atasi sakit maag yang tak kunjung sembu...,usus halus ataupun kerongkongan yang dapat dis...,Kamu adalah asisten medis cerdas yang membantu...,0.681906,0.692555,0.687189
3,solusi untuk mengatasi konstipasi,solusi untuk mengatasi konstipasi. dok kalo ba...,konstipasi adalah keadaan tidak dapat buang ai...,Kamu adalah asisten medis cerdas yang membantu...,0.622037,0.652146,0.636736
4,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. s...","penyebab sakit perut, mual dan bersendawa mung...",Kamu adalah asisten medis cerdas yang membantu...,0.673499,0.677910,0.675697
...,...,...,...,...,...,...,...
97,Muntah-muntah pada penderita infeksi lambung a...,Muntah-muntah pada penderita infeksi lambung a...,lambung adalah organ yang vital fungsinya dala...,Kamu adalah asisten medis cerdas yang membantu...,0.696169,0.702389,0.699265
98,"Demam tinggi, mual, dan tidak nafsu makan","Demam tinggi, mual, dan tidak nafsu makan. do...","kolera, disentri dulu. radang usus = kolitis ,...",Kamu adalah asisten medis cerdas yang membantu...,0.639596,0.665616,0.652347
99,Perbedaan serangan jantung dan lambung,Perbedaan serangan jantung dan lambung. Dok ma...,"namun, untuk membedakan apakah serangan jantun...",Kamu adalah asisten medis cerdas yang membantu...,0.652509,0.717806,0.683602
100,Penyebab BAB berdarah dan mengenai pemeriksaan...,Penyebab BAB berdarah dan mengenai pemeriksaan...,anal seks merupakan salah satu perbuatan beres...,Kamu adalah asisten medis cerdas yang membantu...,0.636416,0.654311,0.645240


In [ ]:
! zip -r model_base_6.zip /content/eval_basemodel_20250525_075214

updating: content/eval_basemodel_20250525_075214/ (stored 0%)
  adding: content/eval_basemodel_20250525_075214/metrics.json (deflated 46%)
  adding: content/eval_basemodel_20250525_075214/base_model_outputs.csv (deflated 77%)


In [ ]:
!curl -F "reqtype=fileupload" -F "fileToUpload=@/content/model_base_6.zip" https://catbox.moe/user/api.php

https://files.catbox.moe/84q2x9.zip

# Dengan Adapter

In [ ]:
import torch
import gc

# Clear PyTorch cache
torch.cuda.empty_cache()

# Garbage collection
gc.collect()

815

In [ ]:
MAX_SEQ_LENGTH = 2048
USE_4BIT = True

# === LOAD LoRA MODEL === #
model2, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "farwew/Med-QA",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,
    load_in_4bit = USE_4BIT,
)
FastLanguageModel.for_inference(model2)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.5.7: Fast Gemma2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/216M [00:00<?, ?B/s]

Unsloth 2025.5.7 patched 42 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "mencegah mual",
    "Anak saya mengalami mual, bagaimana mencegah mual?",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model2.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

You have set `use_cache` to `False`, but cache_implementation is set to dynamic. cache_implementation will have no effect.


<bos>Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
mencegah mual

### Detail Pertanyaan:
Anak saya mengalami mual, bagaimana mencegah mual?

### Jawaban:


AUTOTUNE bmm(16x82x256, 16x256x82)
  bmm 0.0154 ms 100.0% 
  triton_bmm_1 0.0154 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=128, BLOCK_M=32, BLOCK_N=32, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_2 0.0174 ms 88.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=32, BLOCK_N=64, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_7 0.0174 ms 88.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=64, BLOCK_M=64, BLOCK_N=64, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=8
  triton_bmm_6 0.0195 ms 78.9% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=64, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_3 0.0205 ms 75.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=32, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=8
  triton_bmm_4 0.0215 ms 71.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=128, BLOCK_M=64, BLOCK_N=32, EVEN_K=False, GROUP_M=8, num_stages

Mual merupakan suatu sensasi tidak nyaman pada perut bagian atas yang disertai dengan dorongan untuk muntah. Mual dapat disebabkan oleh berbagai hal, diantaranya : penyakit asam lambung infeksi saluran pencernaan keracunan makanan vertigo mabuk perjalanan efek samping obat-obatan tertentu stres psikologis dan lain-lain untuk mencegah mual, anda dapat melakukan beberapa hal berikut ini : makan secara teratur dengan porsi kecil namun sering hindari makanan yang terlalu pedas, asam, berlemak, dan berminyak hindari minuman berkafein, bersoda, dan beralkohol hindari berbaring setelah makan hindari stres psikologis hindari paparan asap rokok dan debu kendaraan jika mual yang dialami tidak kunjung membaik, sebaiknya anda memeriksakan diri ke dokter agar dapat dilakukan pemeriksaan lebih lanjut untuk mengetahui penyebabnya sehingga dapat diberikan penanganan yang tepat.<eos>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
    "Selalu tersedak setiap makan dan minum",
    "Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...",
    ""
)
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model2.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Selalu tersedak setiap makan dan minum

### Detail Pertanyaan:
Selalu tersedak setiap makan dan minum. Dok saya mau tanya... Kenapa kalau saya makan atau minum selalu tersedak.. Akhirnya susah nafas.. Dada sangat sesak.... Hampir setiap makan saya tersedak...

### Jawaban:
Tersedak merupakan suatu kondisi dimana makanan atau minuman masuk ke saluran pernapasan. Hal ini dapat terjadi karena beberapa hal, seperti: makan terlalu cepat makan sambil berbicara makan sambil berbaring makan makanan yang terlalu keras atau terlalu kecil (misalnya kacang-kacangan) minum minuman yang terlalu kental atau terlalu dingin (misalnya susu) merokok atau minum minuman beralkohol terlalu banyak berbicara atau tertawa terlalu keras penyakit asam 

In [ ]:
# === IMPORT TAMBAHAN === #
import bert_score
import pandas as pd
import json
from datetime import datetime
import os

# === LOAD METRIK EVALUASI === #
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_outputs = []
reference_outputs = []
all_inputs = []  # Untuk menyimpan input
all_instructions = []  # Untuk menyimpan instruction

# === PARAMETER === #
batch_size = 1  # Sesuaikan dengan kapasitas GPU
MAX_DISPLAY = 10
displayed = 0

# === TIMESTAMP UNTUK NAMA FILE === #
timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"evaluation_results_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
stratified_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'topic_set', 'topics', '__index_level_0__'],
    num_rows: 100
})

In [ ]:
print("Total parameters:", model2.num_parameters())
# print("Trainable parameters:", model.get_nb_trainable_parameters())


Total parameters: 9295724032


In [ ]:
num = 1
for example in tqdm(stratified_dataset, desc="Evaluating FT tune Model2"):
    prompt = alpaca_prompt.format(example["instruction"], example["input"], "")
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model2.generate(
            **input_ids,
            max_new_tokens=1024,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    response = decoded.split("### Response:")[-1].strip()

    if num < 10:
        print(prompt)
        print()
        print('response:',response)

    generated_outputs.append(response)
    reference_outputs.append(example["output"])
    all_instructions.append(example["instruction"])
    all_inputs.append(example["input"])
    num+=1

Evaluating FT tune Model2:   1%|          | 1/100 [00:44<1:14:12, 44.97s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Solusi atasi sakit maag yang tak kunjung sembuh

### Detail Pertanyaan:
Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1,5 kilo, tidak memiliki nafsu makan sama sekali, sering sendawa, nyeri di ulu hati dan perut bagian kanan. Kemarin saya juga sempat terlambat makan siang dan malam, akibatn

Evaluating FT tune Model2:   2%|▏         | 2/100 [01:41<1:24:28, 51.72s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
solusi untuk mengatasi konstipasi

### Detail Pertanyaan:
solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?

### Jawaban:
Perlu anda ketahui sebelumnya bahwa konstipasi

Evaluating FT tune Model2:   3%|▎         | 3/100 [02:40<1:29:07, 55.12s/it]

...

Evaluating FT tune Model2:   8%|▊         | 8/100 [05:25<54:59, 35.87s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
sakit perut sebelah kiri bawah

### Detail Pertanyaan:
sakit perut sebelah kiri bawah. Dok, saya mau tanya. Saya sudah menikah dan berusia 21 tahun. Sudah hampir 2minggu lebih perut sebelah kiri bawah sakit. Terhitung dari selesai saya haid tanggal 30 maret 2016. Sakitnya sampai ke pinggang dan tanda tanda nya seperti akan haid. Karena pada saya pun terasa nyeri dan cenderung lebih sensitif. Di minggu ketiga ini sakitnya terasa ngilu sampai ke paha kiri. Bahkan kadang panggul panggul pun sakit. Itu kenapa ya dok?apakah saya hamil? Terima kasih

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sa

Evaluating FT tune Model2:   9%|▉         | 9/100 [06:04<56:09, 37.03s/it]

Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>

### Keluhan:
Nyeri Ulu Hati Pasca Operasi Usus buntu

### Detail Pertanyaan:
Nyeri Ulu Hati Pasca Operasi Usus buntu. sudah 4 Bulan yang lalu saya di operasi usus buntu (Operasi Laparotomi) minggu minggu ini perut saya aga kurang enak dok. tadi pagi setelah BAB perut saya kram,melilit,perih....dan nyeri Ulu ati, nyerinya timbul tenggelam...tapi setelah minum obat maag (Omeprazol) sakitnya mereda tapi masih aga kurang enak perutnya itu penyebabnya apa ya dok? apa karena Maag atau ada iritasi di Usus? atau ada hubungannya dengan operasi usus buntu 4 bulan yang lalu?

### Jawaban:


response: Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampu

Evaluating FT tune Model2: 100%|██████████| 100/100 [1:22:06<00:00, 49.26s/it]


In [ ]:
# === Skor Evaluasi === #
P, R, F1 = bert_score.score(generated_outputs, reference_outputs, lang="id", device="cuda")
bleu_score = bleu.compute(predictions=generated_outputs, references=[[r] for r in reference_outputs])
rouge_score = rouge.compute(predictions=generated_outputs, references=reference_outputs)

timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
output_dir = f"eval_basemodel_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# === Simpan Hasil Evaluasi === #
df = pd.DataFrame({
    "instruction": all_instructions,
    "input": all_inputs,
    "reference": reference_outputs,
    "generated": generated_outputs,
    "bert_precision": P.tolist(),
    "bert_recall": R.tolist(),
    "bert_f1": F1.tolist(),
})

df.to_csv(f"{output_dir}/fine_tuned_outputs.csv", index=False)

metrics = {
    "bert_score": {
        "precision": round(P.mean().item() * 100, 2),
        "recall": round(R.mean().item() * 100, 2),
        "f1": round(F1.mean().item() * 100, 2)
    },
    "bleu": round(bleu_score["bleu"] * 100, 2),
    "rouge": {
        "rouge1": round(rouge_score["rouge1"] * 100, 2),
        "rouge2": round(rouge_score["rouge2"] * 100, 2),
        "rougeL": round(rouge_score["rougeL"] * 100, 2)
    }
}

with open(f"{output_dir}/metrics.json", "w") as f:
    import json; json.dump(metrics, f, indent=4)

print("=== EVALUASI Fine tune MODEL SELESAI ===")
print(metrics)

=== EVALUASI Fine tune MODEL SELESAI ===
{'bert_score': {'precision': 65.74, 'recall': 68.93, 'f1': 67.27}, 'bleu': 3.48, 'rouge': {'rouge1': np.float64(31.04), 'rouge2': np.float64(6.54), 'rougeL': np.float64(14.02)}}


In [ ]:
! zip -r moedel_base_3.zip /content/eval_basemodel_20250525_025234

updating: content/eval_basemodel_20250525_025234/ (stored 0%)
  adding: content/eval_basemodel_20250525_025234/metrics.json (deflated 46%)
  adding: content/eval_basemodel_20250525_025234/base_model_outputs.csv (deflated 68%)


In [ ]:
! zip -r moedel_ft_6.zip /content/eval_basemodel_20250525_102353

updating: content/eval_basemodel_20250525_102353/ (stored 0%)
updating: content/eval_basemodel_20250525_102353/fine_tuned_outputs.csv (deflated 76%)
updating: content/eval_basemodel_20250525_102353/metrics.json (deflated 45%)


In [ ]:
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 4.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.21 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 16%|█▌        | 5/32 [00:00<00:02, 10.64it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:55<00:00,  3.60s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00003.bin...
Unsloth: Saving model/pytorch_model-00002-of-00003.bin...
Unsloth: Saving model/pytorch_model-00003-of-00003.bin...
Done.


Unsloth: You are pushing to hub, but you passed your HF username = hf.
We shall truncate hf/model to model


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.04 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [01:47<00:00,  3.37s/it]


Unsloth: Saving to organization with address hf/model
Unsloth: Saving tokenizer... Done.
Unsloth: Saving to organization with address hf/model
Unsloth: Saving hf/model/pytorch_model-00001-of-00003.bin...
Unsloth: Saving hf/model/pytorch_model-00002-of-00003.bin...
Unsloth: Saving hf/model/pytorch_model-00003-of-00003.bin...
Unsloth: Uploading all files... Please wait...


RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-683297d4-294fcc9f4169643d40a54ee1;9cd55209-8300-4cff-9512-7041219b2a18)

Repository Not Found for url: https://huggingface.co/api/models/hf/model/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.

In [ ]:
if True: model.push_to_hub_merged("farwew/model_3", tokenizer, save_method = "merged_16bit", token = "hf_REMOVED_BY_SCRIPT")

Unsloth: You are pushing to hub, but you passed your HF username = farwew.
We shall truncate farwew/model_3 to model_3


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.56 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 41%|████      | 13/32 [00:52<01:15,  3.97s/it]